# Full Archive Search

- Coppying form this github page https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/master/Full-Archive-Search/full-archive-search.py

In [1]:
import time
import pandas as pd
import requests
import os
import json

# Import keys from file not commited to GitHub to keep my credentials secret
from setup_api import bearer_token

In [2]:
# Setting up functions needed for search

search_url = "https://api.twitter.com/2/tweets/search/all"

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", search_url, headers=headers, params=params)
    print(response.status_code)
    if response.status_code != 200:
        print(response.status_code)
        raise Exception(response.status_code, response.text)
    return response.json()


# Turn into function for a given user

Got it to work here, try to functionise it

In [3]:

# Get all results from one user

# TODO: Documentation
def users_tweets(user, start_time, end_time):
    '''
    users - a list of twitter user
    start_time - start date + time we want #TODO: Implement
    end_time - end date + time we want   #TODO: Implement
    '''
    num_runs = 0
    all_results = []

    result_count = 100  # initilise this as a non-zero number
                        # will be filled with respnses from json later 
                        # can get a max of 100 records at a time
                        # so if the results arnt 100 we ran out of things to get

    while result_count == 100:

        time.sleep(4)

        if num_runs ==0:
            # No next_token parameter
            query_params = { #"query":"from:CNNChile lang:es",
                    'query': f'(from:{user} -is:retweet) ',
                    'tweet.fields': 'author_id,id,created_at',
                    "start_time":"2019-10-01T07:20:50.52Z",
                    "end_time":"2019-11-01T07:20:50.52Z",
                    "max_results": "100"
            }
        else:
            # with next_token
            query_params = { #"query":"from:CNNChile lang:es",
                    'query': f'(from:{user} -is:retweet) ',
                    'tweet.fields': 'author_id,id,created_at',
                    "start_time":f"{start_time}",
                    "end_time":f"{end_time}",
                    "max_results": "100",
                    "next_token": results_json['meta']['next_token'] # get token from before: 'b26v89c19zqg8o3fnm91i40gu1gp7vh7j0r6ddrjyhvul'
            }

        headers = create_headers(bearer_token)
        json_response = connect_to_endpoint(search_url, headers, query_params)
        results = json.dumps(json_response, indent=4, sort_keys=True)

        results_json = json.loads(results)

        # append this pagnation to all results
        #all_results.append(results_json)
        result_count = results_json['meta']['result_count']
        num_runs = num_runs + 1
        #print(num_runs, result_count)

        # Only add results if theyre non zero
        if result_count > 0:
            list_of_tweet_dicts = results_json['data']

            print(f"Tweet {len(all_results)} from {list_of_tweet_dicts[0]['created_at']}")

            for tweet in list_of_tweet_dicts:
                all_results.append([user, tweet['created_at'], tweet['id']])

    return all_results

In [4]:
%%time
users_tweets('nigel_farage',"2019-10-01T00:00:00.00Z","2019-11-01T00:00:00.00Z")

200
Tweet 0 from 2019-10-31T19:37:00.000Z
Wall time: 4.55 s


[['nigel_farage', '2019-10-31T19:37:00.000Z', '1189989669620080641'],
 ['nigel_farage', '2019-10-31T18:07:33.000Z', '1189967159176388611'],
 ['nigel_farage', '2019-10-31T16:58:43.000Z', '1189949832821137408'],
 ['nigel_farage', '2019-10-31T11:08:41.000Z', '1189861745009053697'],
 ['nigel_farage', '2019-10-29T20:44:06.000Z', '1189281779603124227'],
 ['nigel_farage', '2019-10-29T18:34:42.000Z', '1189249212447215616'],
 ['nigel_farage', '2019-10-29T18:06:55.000Z', '1189242221121495041'],
 ['nigel_farage', '2019-10-28T19:18:00.000Z', '1188897721085771777'],
 ['nigel_farage', '2019-10-27T18:58:28.000Z', '1188530419635052551'],
 ['nigel_farage', '2019-10-27T17:23:11.000Z', '1188506439742119939'],
 ['nigel_farage', '2019-10-27T10:22:53.000Z', '1188400668983414786'],
 ['nigel_farage', '2019-10-26T22:29:02.000Z', '1188221022601990145'],
 ['nigel_farage', '2019-10-26T14:46:07.000Z', '1188104526911496192'],
 ['nigel_farage', '2019-10-26T11:55:50.000Z', '1188061673992667137'],
 ['nigel_farage', '2

In [5]:
%%time
CNNChile_tweets = users_tweets('CNNChile',"2019-10-01T00:00:00.00Z","2019-10-07T00:00:00.00Z")

200
Tweet 0 from 2019-11-01T01:51:01.000Z
200
Tweet 100 from 2019-10-06T23:15:00.000Z
200
Tweet 200 from 2019-10-04T16:49:58.000Z
200
Tweet 300 from 2019-10-03T13:25:11.000Z
200
Tweet 400 from 2019-10-01T22:39:27.000Z
Wall time: 22.9 s


In [6]:

# Function to get all tweet ID's from a given date range for a list of users

def get_list_users_tweets(users, start_date, end_date):
    '''
    users - a list of twitter user
    start_date - start date + time we want
    end_date - end date + time we want
    '''

    res = []

    for user in users:
        
        print('-'*30)
        print(f"User: {user} Number: {users.index(user)+1} of {len(users)}")
        print('-'*30)
        print()
        # Append results of function to results
        # Extend will insure everying is in the same list
        # Append would create a list of lists
        res.extend( users_tweets(user, start_date, end_date ) )

    #Convert results from list of list to DataFrame
    tweets_df = pd.DataFrame.from_records(res, columns=['User', 'TweetCreated', 'TweetId'])

    return tweets_df


# Chilian News sources

### So far these are the sources that have been identified as potentially interesting:

- El Mercurio (@ElMercurio_cl) / Twitter
- La Tercera (@latercera) / Twitter
- laSegunda (@La_Segunda) / Twitter
- Las Últimas Noticias (@lun) / Twitter
- La Cuarta (@lacuarta) / Twitter
- Meganoticias (@meganoticiascl) / Twitter
- Canal 13 (@canal13) / Twitter
- TVN (@TVN) / Twitter
- 24 Horas (@24HorasTVN) / Twitter
- La Nación Chile (@nacioncl) / Twitter
- Diario Financiero (@DFinanciero) / Twitter
- Chilevisión (@chilevision) / Twitter
- UCV Radio 103.5 FM (@ucvradio) / Twitter
- LaRed (@LaRedTV) / Twitter
- Cooperativa (@Cooperativa) / Twitter
- Radio Pudahuel (@RadioPudahuel) / Twitter
- BioBioChile (@biobio) / Twitter
- El Conquistador 91.3 (@FMConquistador) / Twitter
- EL INFORMADORCHILE🇨🇱 (@INFORMADORCHILE) / Twitter
- CNN Chile (@CNNChile) / Twitter
- CHV Noticias (@CHVNoticias) / Twitter
- El Mostrador (@elmostrador) / Twitter
- Publimetro (@PublimetroChile) / Twitter
- El Desconcierto (@eldesconcierto) / Twitter
- El Dínamo (@el_dinamo) / Twitter
- El Ciudadano (@El_Ciudadano) / Twitter
- El Líbero (@elliberocl) / Twitter
- (4) Diario El Observador (@eo_enlinea) / Twitter




In terms of the time period, if you could look between October 18, 2019 and November 18, 2019, that would be some cool gravy.




In [7]:
%%time

chile_news_users = ['ElMercurio_cl',
                    'latercera',
                    'La_Segunda',
                    'lun',
                    'lacuarta',
                    'meganoticiascl',
                    'canal13',
                    'TVN',
                    '24HorasTVN',
                    'nacioncl',
                    'DFinanciero',
                    'chilevision',
                    'ucvradio',
                    'LaRedTV',
                    'Cooperativa',
                    'RadioPudahuel',
                    'biobio',
                    'FMConquistador',
                    'INFORMADORCHILE',
                    'CNNChile',
                    'CHVNoticias',
                    'elmostrador',
                    'PublimetroChile',
                    'eldesconcierto',
                    'el_dinamo',
                    'El_Ciudadano',
                    'elliberocl',
                    'eo_enlinea']

start_date = "2019-10-01T00:00:00.00Z"
end_date = "2019-11-01T00:00:00.00Z"

# Try getting just the dates we want in 2019
results = get_list_users_tweets(chile_news_users, start_date, end_date)


------------------------------
User: ElMercurio_cl Number: 1 of 28
------------------------------

200
Tweet 0 from 2019-11-01T02:50:00.000Z
200
Tweet 100 from 2019-10-30T18:00:00.000Z
200
Tweet 200 from 2019-10-28T20:00:00.000Z
200
Tweet 300 from 2019-10-27T12:50:00.000Z
200
Tweet 400 from 2019-10-25T00:00:00.000Z
200
Tweet 500 from 2019-10-22T21:00:00.000Z
200
Tweet 600 from 2019-10-20T16:03:14.000Z
200
Tweet 700 from 2019-10-18T17:00:01.000Z
200
Tweet 800 from 2019-10-16T20:00:00.000Z
200
Tweet 900 from 2019-10-14T22:20:00.000Z
200
Tweet 1000 from 2019-10-13T10:30:00.000Z
200
Tweet 1100 from 2019-10-11T00:30:00.000Z
200
Tweet 1200 from 2019-10-09T10:30:00.000Z
200
Tweet 1300 from 2019-10-07T12:40:00.000Z
200
Tweet 1400 from 2019-10-05T12:45:00.000Z
200
Tweet 1500 from 2019-10-03T13:00:00.000Z
200
Tweet 1600 from 2019-10-01T14:00:00.000Z
------------------------------
User: latercera Number: 2 of 28
------------------------------

200
Tweet 0 from 2019-11-01T07:20:00.000Z
200
Tweet 1

In [8]:
results

,User,TweetCreated,TweetId
0,ElMercurio_cl,2019-11-01T02:50:00.000Z,1190098635121283072
1,ElMercurio_cl,2019-11-01T02:40:00.000Z,1190096118329135104
2,ElMercurio_cl,2019-11-01T02:30:00.000Z,1190093602464129024
3,ElMercurio_cl,2019-11-01T02:20:00.000Z,1190091085407825921
4,ElMercurio_cl,2019-11-01T02:10:00.000Z,1190088568707923969
...,...,...,...
62690,eo_enlinea,2019-10-01T13:31:45.000Z,1179026113860902913
62691,eo_enlinea,2019-10-01T13:15:09.000Z,1179021934601920512
62692,eo_enlinea,2019-10-01T13:01:59.000Z,1179018624062894080
62693,eo_enlinea,2019-10-01T12:55:34.000Z,1179017007532036098


In [9]:
results.to_csv('reults.csv')

In [11]:
results.head()

,User,TweetCreated,TweetId
0,ElMercurio_cl,2019-11-01T02:50:00.000Z,1190098635121283072
1,ElMercurio_cl,2019-11-01T02:40:00.000Z,1190096118329135104
2,ElMercurio_cl,2019-11-01T02:30:00.000Z,1190093602464129024
3,ElMercurio_cl,2019-11-01T02:20:00.000Z,1190091085407825921
4,ElMercurio_cl,2019-11-01T02:10:00.000Z,1190088568707923969


In [16]:
len(results)

62695

In [15]:
results['User'].value_counts()

Cooperativa        12083
latercera           5822
24HorasTVN          5450
meganoticiascl      4875
PublimetroChile     4613
biobio              4210
DFinanciero         2240
CNNChile            2155
elmostrador         2089
eldesconcierto      2047
El_Ciudadano        1908
lacuarta            1637
ElMercurio_cl       1624
CHVNoticias         1538
el_dinamo           1414
TVN                 1375
canal13             1311
elliberocl          1169
nacioncl            1066
LaRedTV              766
ucvradio             692
chilevision          645
lun                  575
RadioPudahuel        447
eo_enlinea           382
FMConquistador       366
INFORMADORCHILE      196
Name: User, dtype: int64